In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Only for collab

In [2]:
!pip install -U transformers datasets evaluate accelerate
!pip install scikit-learn
!pip install tensorboard

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/ad/67/b4d6a51dcaf988cb45b31e26c6e33fb169fe34ba5fb168b086309bd7c028/transformers-4.37.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.3 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/ec/93/454ada0d1b289a0f4a86ac88dbdeab54921becabac45da3da787d136628f/datasets-2.16.1-py3-none-any.whl.metadata
  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/a6/b9/44623bdb05595481107153182e7f4b9f2ef9d3b674938ad13842054dcbd8/accelerate-0.26.1-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/p

In [3]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline,
)

import evaluate
import glob
import numpy as np

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
import pandas as pd
df = pd.read_csv('/kaggle/input/dark-buster/dark.csv')

In [5]:
df.head()

,text,label
0,FLASH SALE | LIMITED TIME ONLY Shop Now,1
1,Pillowcases & Shams,0
2,Write a review,0
3,"To start your return, simply click on the foll...",0
4,newsletter signup (privacy policy),0


In [6]:
BATCH_SIZE = 32
NUM_PROCS = 32
LR = 0.00005
EPOCHS = 10
MODEL = 'bert-base-uncased'
OUT_DIR = '/kaggle/working/models'

In [7]:
df['label'] = df['label'].astype('int')

# Sample indices for train, validation, and test sets
train_indices = df.sample(frac=0.8, random_state=42).index
valid_indices = df.sample(frac=0.1, random_state=42).index
test_indices = df.index.difference(train_indices).difference(valid_indices)

# Filter rows based on indices
train_dataset = df[df.index.isin(train_indices)][['text', 'label']]
valid_dataset = df[df.index.isin(valid_indices)][['text', 'label']]
test_dataset = df[df.index.isin(test_indices)][['text', 'label']]


In [8]:
print(train_dataset.iloc[1])

text     Write a review
label                 0
Name: 2, dtype: object


In [9]:
def preprocess_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
    )

In [10]:
from datasets import Dataset

# Preprocess the datasets
train_dataset = Dataset.from_pandas(train_dataset)
valid_dataset = Dataset.from_pandas(valid_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

In [11]:
id2label = {
    0: "not_dark",
    1: "urgency",
    2: "scarcity",
    3: "mis_direction",
    4: "social_proof",
    5: "sneaking",
    6: "obstruction",
    7: "forced_action"
}

label2id = {label: idx for idx, label in id2label.items()}

In [12]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
tokenized_train = train_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=BATCH_SIZE,
    num_proc=NUM_PROCS
)

tokenized_valid = valid_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=BATCH_SIZE,
    num_proc=NUM_PROCS
)

tokenized_test = test_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=BATCH_SIZE,
    num_proc=NUM_PROCS
)

Map (num_proc=32):   0%|          | 0/1885 [00:00<?, ? examples/s]

Map (num_proc=32):   0%|          | 0/236 [00:00<?, ? examples/s]

Map (num_proc=32):   0%|          | 0/471 [00:00<?, ? examples/s]

In [14]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [15]:
tokenized_sample = preprocess_function(train_dataset[0])
print(tokenized_sample)

{'input_ids': [101, 5956, 5096, 1064, 3132, 2051, 2069, 4497, 2085, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


metrics

In [16]:
accuracy = evaluate.load('accuracy')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [17]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL,
    num_labels=8,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=OUT_DIR,
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=3,
    report_to='tensorboard',
    fp16=True  # Set to True for mixed-precision training
)


In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

history = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.134723,0.970339
2,No log,0.093987,0.978814
3,No log,0.070301,0.983051
4,No log,0.026253,0.995763
5,No log,0.006925,1.000000
6,No log,0.002510,1.000000
7,No log,0.001880,1.000000
8,No log,0.001652,1.000000
9,0.139600,0.001502,1.000000
10,0.139600,0.001459,1.000000


In [20]:
trainer.evaluate(tokenized_test)

{'eval_loss': 0.1710023730993271,
 'eval_accuracy': 0.9639065817409767,
 'eval_runtime': 1.0331,
 'eval_samples_per_second': 455.906,
 'eval_steps_per_second': 14.519,
 'epoch': 10.0}

In [21]:
%cd /kaggle/working
from IPython.display import FileLinks

# Assuming that the '*' in your original code is meant to be a placeholder
# Replace 'models' with the actual filename or pattern you want to link
FileLinks('models')


/kaggle/working


models/checkpoint-472/
  tokenizer_config.json
  trainer_state.json
  model.safetensors
  vocab.txt
  scheduler.pt
  tokenizer.json
  training_args.bin
  special_tokens_map.json
  optimizer.pt
  config.json
  rng_state.pth
models/checkpoint-531/
  tokenizer_config.json
  trainer_state.json
  model.safetensors
  vocab.txt
  scheduler.pt
  tokenizer.json
  training_args.bin
  special_tokens_map.json
  optimizer.pt
  config.json
  rng_state.pth
models/checkpoint-590/
  tokenizer_config.json
  trainer_state.json
  model.safetensors
  vocab.txt
  scheduler.pt
  tokenizer.json
  training_args.bin
  special_tokens_map.json
  optimizer.pt
  config.json
  rng_state.pth
models/runs/Jan24_17-39-39_42a8ffc47c3f/
  events.out.tfevents.1706117979.42a8ffc47c3f.26.0
  events.out.tfevents.1706118148.42a8ffc47c3f.26.1

In [22]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/models/checkpoint-590")
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Move the model to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Assuming train_dataset is defined and contains your training data
# demo = train_dataset[1]['text']
demo = "5 items left in the stock"

# Tokenize on the same device as the model
inputs = tokenizer(demo, return_tensors='pt')

# Move the input tensors to the same device as the model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Make sure to use the correct function to get predictions based on your specific use case (e.g., binary classification, multi-class classification)
# Replace 'classify' with the appropriate function for prediction
res = model(**inputs)

# Further processing of the result based on your specific use case
# For example, if it's binary classification, you might want to apply a sigmoid activation function:
# res = torch.sigmoid(res.logits)

# Assuming res is a tensor, you can get the predicted class index
predicted_class_index = torch.argmax(res.logits).item()

# Get the predicted label using id2label
predicted_label = id2label.get(predicted_class_index, f'Unknown Label ({predicted_class_index})')

# Print the original sentence, predicted label, and class
print("Original Sentence:", demo)
print("Predicted Label:", predicted_label)
print("Predicted Class Index:", predicted_class_index)


Original Sentence: 5 items left in the stock
Predicted Label: scarcity
Predicted Class Index: 2


In [24]:
from huggingface_hub import notebook_login
notebook_login()

In [26]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# Load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/models/checkpoint-590")
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Save the model and tokenizer to a directory
model.save_pretrained("/kaggle/working/final")
tokenizer.save_pretrained("/kaggle/working/final")

# Push the model to the Hugging Face Model Hub
model.push_to_hub("sekhharr/hackathon_v1")

# Push the tokenizer to the Hugging Face Model Hub
tokenizer.push_to_hub("sekhharr/hackathon_v1")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sekhharr/hackathon_v1/commit/a8810c59c7a34b663b6eda93deef9e62facda042', commit_message='Upload tokenizer', commit_description='', oid='a8810c59c7a34b663b6eda93deef9e62facda042', pr_url=None, pr_revision=None, pr_num=None)

In [27]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("sekhharr/hackathon_v1")
tokenizer = AutoTokenizer.from_pretrained('sekhharr/hackathon_v1')

# Move the model to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Assuming train_dataset is defined and contains your training data
# demo = train_dataset[1]['text']
demo = "5 items left in the stock"

# Tokenize on the same device as the model
inputs = tokenizer(demo, return_tensors='pt')

# Move the input tensors to the same device as the model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Make sure to use the correct function to get predictions based on your specific use case (e.g., binary classification, multi-class classification)
# Replace 'classify' with the appropriate function for prediction
res = model(**inputs)

# Further processing of the result based on your specific use case
# For example, if it's binary classification, you might want to apply a sigmoid activation function:
# res = torch.sigmoid(res.logits)

# Assuming res is a tensor, you can get the predicted class index
predicted_class_index = torch.argmax(res.logits).item()

# Get the predicted label using id2label
predicted_label = id2label.get(predicted_class_index, f'Unknown Label ({predicted_class_index})')

# Print the original sentence, predicted label, and class
print("Original Sentence:", demo)
print("Predicted Label:", predicted_label)
print("Predicted Class Index:", predicted_class_index)


config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Original Sentence: 5 items left in the stock
Predicted Label: scarcity
Predicted Class Index: 2
